In [1]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from matplotlib.animation import FuncAnimation
import os

In [5]:
save_path = r'C:\Users\leopo\OneDrive - UT Cloud\Uni\Semester_8\BA_mit_Git\BA_Plots\Qutip'
os.makedirs(os.path.dirname(save_path), exist_ok=True)

In [6]:
########################################                 Define constants                   #############################################
N = 30
fixed_lam   = 1
fixed_gamma = 1
fixed_dist  = 0.3 * fixed_lam

t_max    = 40
t_size   = 100
times  = np.linspace(0, t_max, t_size) * fixed_gamma

In [16]:
########################################               define the geometry                 #############################################
def dipole_vector(phi = 0):
    """Returns the dipole vector given its orientation by angle phi."""
    dipole = np.array([np.cos(phi), np.sin(phi), 0])
    return dipole

def z_rotation(angle):
    return np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle), np.cos(angle), 0],
        [0, 0, 1]])

def ring_positions(distance = fixed_dist, N = N):
    Pos = np.zeros((N, 3))
    dphi = 2 * np.pi / N
    radius = distance / 2 / np.sin(np.pi / N)
    helper = np.array([radius, 0, 0])
    for i in range(N):
        rotation_matrix = np.linalg.matrix_power(z_rotation(dphi), i)
        Pos[i] = np.matmul(rotation_matrix, helper)
    return Pos

In [8]:
########################################             create the Hamiltonean                 #################################################
def Green_tensor(r_a, r_b, k_a):
    r_ab = r_b - r_a
    abs_r_ab = np.linalg.norm(r_ab)
    kappa = k_a * abs_r_ab
    return (np.exp(1j * kappa) / (4 * np.pi * kappa ** 2 * abs_r_ab)
                    * ((kappa ** 2 + 1j * kappa - 1) * np.eye(3)
                       + (- kappa ** 2 - 3 * 1j * kappa + 3)
                       * np.outer(r_ab, r_ab) / (abs_r_ab ** 2)))

def Gamma_matrix(distance, dipoles, lam, gamma):
    positions = ring_positions(distance)
    G_matrix = np.zeros((N, N), dtype=complex)
    for a in range(N):
        for b in range(N):
            G_matrix[a, b] = gamma
            r_a, r_b = positions[a], positions[b]
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                k_a = 2 * np.pi / lam
                result = (6 * np.pi * gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b, k_a), d_b.T)))
                G_matrix[a, b] = np.imag(result)
    return G_matrix

def V_matrix(distance, dipoles, lam, gamma):
    positions = ring_positions(distance, N)
    V_matrix = np.zeros((N, N), dtype=complex)
    for a in range(N):
        for b in range(N):
            r_a, r_b = positions[a], positions[b]
            V_matrix[a, b] = 0
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                k_a = 2 * np.pi / lam
                result = (3 * np.pi * gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b, k_a), d_b.T)))
                V_matrix[a, b] = np.real(result)
    return V_matrix
    
def H_eff(distance = fixed_dist, dipoles = [dipole_vector() for _ in range(N)], lam=fixed_lam, gamma=fixed_gamma):
    G = Gamma_matrix(distance, dipoles, lam, gamma)
    V = V_matrix(distance, dipoles, lam, gamma)
    return Qobj(V) - 1j / 2 * Qobj(G)

In [17]:
# Define the position indices with periodic boundary conditions
center_index = N
k_s = np.pi / fixed_dist
sigma = 0.1 * k_s
x_j = (np.arange(N) - center_index) * fixed_dist
x_j = np.mod(x_j + N//2 * fixed_dist, N * fixed_dist) - N//2 * fixed_dist
# Calculate the Gaussian wave packet in real space with periodic boundary conditions
coefficients = np.sqrt(sigma / np.sqrt(2 * np.pi)) * np.exp(-1j * k_s * x_j) * np.exp(-sigma**2 * x_j**2)
wave_packet = sum(coeff * basis(N, (j+center_index)%N) for j, coeff in enumerate(coefficients))
psi0 = wave_packet.unit()

# Plot the real and imaginary parts of the wave packet coefficients
plt.figure(figsize=(10, 6))
plt.plot(x_j, coefficients.real, 'bo-', label='Real Part')
plt.plot(x_j, coefficients.imag, 'ro-', label='Imaginary Part')
plt.xlabel('Position $x_j$')
plt.ylabel('Coefficient')
plt.title('Gaussian Wave Packet in Real Space with Periodic Boundary Conditions')
plt.legend()
plt.grid(True)
plt.show()

In [18]:
from scipy.optimize import minimize
def survival_probabilities(distance, times, Psi_0=psi0): #
    positions = ring_positions(distance)
    dipoles = np.zeros_like(positions)
    for i in range(N):
        dipoles[i] = [0,0,1]#dipole_vector(np.arctan2(positions[i,1], positions[i,0])+np.pi/2) 
    H = H_eff(distance, dipoles)
    coeffs_sq_mods = np.zeros((len(times), N))
    P_surs = np.zeros(len(times))
    
    for t_idx, t in enumerate(times):
        U = (-1j * H * t).expm()
        Psi_t = (U * Psi_0).full().flatten()
        Probs = np.abs(Psi_t)**2
        coeffs_sq_mods[t_idx, :] = Probs
        P_surs[t_idx] = Probs.sum()
    return coeffs_sq_mods, P_surs

# Define an optimization function that returns the negative of survival probability
def optimization_target(params):
    _, res = survival_probabilities(params[0], [times[-1]])
    return -res[0]

# Initial guess for the distance
initial_distance = 0.234 * fixed_lam

# Perform optimization
result = minimize(optimization_target, [initial_distance], method='SLSQP', options={'maxiter': 1000})

optimized_distance = result.x[0]

print("Optimized distance:")
print(optimized_distance)

In [14]:
y1, y2 = survival_probabilities(optimized_distance, times, Psi_0=psi0)

In [15]:
# Plot atoms with their dipole moments
positions = ring_positions(optimized_distance, N)
# TODO implement this everywhere
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))

# Plot the evolution of atom state probabilities
im = ax1.imshow(y1.T, aspect='auto', origin='lower', cmap='viridis', extent=[0, t_max, 0, N-1])
ax1.set_xlabel('Time')
ax1.set_ylabel('Atom index')
ax1.set_title('Time evolution of atom state probabilities')
fig.colorbar(im, ax=ax1, label='Probability')

# Plot atoms with their dipole moments
ax2.scatter(positions[:, 0], positions[:, 1], color='blue', s=50, label='Atoms')
for i in range(N):
    ax2.arrow(positions[i, 0], positions[i, 1], dipoles[i][0] * 0.2, dipoles[i][1] * 0.2, head_width=0.5 * 0.01, head_length=0.5 * 0.01, fc='red', ec='red')
ax2.set_title('Atom Positions and Dipole Moments')
ax2.set_xlabel('X Position')
ax2.set_ylabel('Y Position')
ax2.grid(True)
ax2.legend()
ax2.axis('equal')

plt.tight_layout()
plt.show()

# Plot the norms against time
plt.figure(figsize=(12, 2))
plt.plot(times, y2, label='Survival Probability')
plt.plot(times, np.exp(-times), 'r--', linewidth=2.5, label=r'exp(-$\gamma t$)')
plt.xlabel('Time') 
plt.ylabel('Norm (Survival Probability)')
plt.title('Survival Probability vs. Time for Dipole Configuration with $\phi = \pi/2$')
plt.legend()
plt.grid(True)
plt.show()